In [41]:
import pandas as pd
import random
import os

populationSize = 8
mutationRate = 0.2
eliteSchedules = 1
tournamentSelection = 2


class Data:  # Contains all data in lists
    def __init__(self):
        self.rooms = [];
        self.examtime = [];

        '''
        df2 = pd.read_excel('courses.csv.xlsx')
        self.Courses = df2.to_numpy().tolist()

        df = pd.read_excel('teachers.csv.xlsx')
        self.Teachers = df.to_numpy().tolist()

        df1 = pd.read_excel('studentNames.csv.xlsx')
        self.StudentNames = df1.to_numpy().tolist()

        df3 = pd.read_excel('studentCourse.csv.xlsx')
        self.StudentCourse = df3.to_numpy().tolist()
        '''

        self.Courses = [['EE227', 'Digital Logic Design'],  # .self indicates instance attributes
                        ['CS211', 'Discrete Structures'],
                        ['SE110', 'Intro to Software Engineering']]

        self.Teachers = [['Aqeel Shahzad'], ['Farah Naz'], ['Hamda Khan'], ['Usman Rashid'], ['Farah Jabeen Awan'],
                         ['Sara Aziz'], ['Gul e Aisha']]

        self.StudentNames = [['Mohamed F Bakaram'], ['Manal Aldaihani'], ['Ibrar Manawer'], ['Adam Flowers'],
                             ['Lilia Kyuchukova']]

        self.StudentCourse = [[0, 'Sam D Edwards', 'CS211'],
                              [1, 'Sheila Hughton', 'EE227'],
                              [2, 'Yasmin Ahmed', 'SE110']]

        room = [["room1", 300], ["room2", 301], ["room3", 302], ["room4", 303], ["room5", 304], ["room6", 305],
                ["room7", 306], ["room8", 307], ["room9", 308], ["room10", 309]]

        timing_exam = [["9:00am - 12:00pm"], ["12:30pm - 3:30pm"], ["11:00am - 2:00pm"], ["1:00pm - 4:00pm"]]

        for i in range(0, len(room)):  # Inserting rooms in list
            self.rooms.append((room[i][0], room[i][1]))
        for i in range(0, len(timing_exam)):  # Inserting 4 timings in list
            self.examtime.append(timing_exam[i])

    def get_courses(self):
        return self.Courses

    def get_teachers(self):
        return self.Teachers

    def get_studentNames(self):
        return self.StudentNames

    def get_studentCourse(self):
        return self.StudentCourse

    def get_rooms(self):
        return self.rooms

    def get_examTime(self):
        return self.examtime



In [42]:
class Exam:  # This class contains data of single exam
    def __init__(self, course, id):
        self.course = course
        self.id = id
        self.studentlist = []
        self.s = d.get_studentCourse()
        self.room = None
        self.time = None
        self.teacher = None

    def retlist(self):  # For all the names of students in a course
        for i in range(0, len(self.s)):
            if self.course == self.s[i][2]:
                self.studentlist.append(self.s[i][1])
        return self.retlist

    def set_teacher(self, teacher): self.teacher = teacher

    def set_room(self, room): self.room = room

    def set_time(self, time): self.time = time

    def get_course(self): return self.course

    def get_room(self): return self.room

    def get_time(self): return self.time

    def get_teacher(self): return self.teacher

    def get_retlist(self):
        self.retlist()
        return self.studentlist

    def get_s(self):
        return self.s



d = Data()


In [43]:
class Schedule:
    def __init__(self):
        self.data = d
        self.exam = []
        self.numOfConflicts = 0
        self.fitness = -1
        self.examNum = 0
        self.isFitnessChanged = True

    def get_exams(self):  # If a different fitness value, get exams
        # self.isFitnessChanged = True
        return self.exam

    def get_NumOfCon(self):
        return self.numOfConflicts

    def get_fitness(self):
        if self.isFitnessChanged:
            self.fitness = self.calculatefitness()  # If fitness changed, update fitness value and return it
            self.isFitnessChanged = False
        return self.fitness

    def initialize(self):

        course = self.data.get_courses()  # Puts all courses

        for i in range(0, len(course)):  # Loop till total number of courses
            self.examNum += 1  # Exam number increments
            new_exam = Exam(course[i], self.examNum)  # New Exam (class) made

            '''
            new_exam.set_room(random.shuffle(d.get_rooms()))
            new_exam.set_teacher(random.shuffle(d.get_teachers()))
            new_exam.set_time(random.shuffle(d.get_examtime()))
            '''

            # ________________CHANGE
            # Assign random room, teacher, time to the exam
            # A random integer will be the subscript for each list
            new_exam.set_time(d.get_examTime()[random.randint(0, 3)])
            new_exam.set_teacher(d.get_teachers()[random.randint(0, 6)])
            new_exam.set_room(d.get_rooms()[random.randint(0, 9)])

            print(self.examNum)
            print(new_exam.get_course())
            print(new_exam.get_time())
            print(new_exam.get_teacher())
            print(new_exam.get_room(), "\n")


            self.exam.append(new_exam)  # Append exam object in list containing all exam objects

        self.calculatefitness()

    def calculatefitness(self):

        self.fitness = -1

        Exams = self.get_exams()

        for i in range(0, len(Exams)):

            for j in range(0, len(Exams)):

                if (Exams[i].get_course() != None):  # Course must have one exam
                    self.fitness += 1

                if(Exams[i].get_retlist()!=Exams[j].get_retlist() and Exams[i].get_time() != Exams[j].get_time()):
                   self.fitness += 1 # Two students not on same time

                if (Exams[i].get_room() != Exams[j].get_room() and Exams[i].get_time() != Exams[j].get_time()):
                    self.fitness += 1  # Time and room not same, 2 exams

                if (Exams[i].get_teacher() != Exams[j].get_teacher() and Exams[i].get_room() != Exams[j].get_room()):
                    self.fitness += 1  # Teacher not same

        print("Fitness:", self.fitness / 10, "\n")
        return self.fitness / 10

        return self



In [44]:
class Population:
    def __init__(self, size):
        self.size = size
        self.data = d
        self.schedules = []  # Contains Schedule classes, which together form the population.
        # Each class contains whole exam schedule, fitness value, conflicts, etc.

        for i in range(0, size):  # Loop makes population of size='size'
            s = Schedule()
            s.initialize()  # This will make an exam schedule and put it in exam list <-(present in schedule class)

            # This will print the details of the first exam of the schedule that is being inserted in schedules list(Population)
            print("DETAILS OF THE FIRST EXAM IN POPULATION'S ELEMENT NUMBER", i)
            print("Course:", s.get_exams()[0].get_course())
            print("Time:", s.get_exams()[0].get_time())
            print("Teacher:", s.get_exams()[0].get_teacher())
            print("Room:", s.get_exams()[0].get_room())

            print("----------------------------------------------------------------")

            self.schedules.append(s)
            # print(self.schedules[0].get_exams[0].get_course())

    def get_schedules(self): return self.schedules


p = Population(5)


1
['EE227', 'Digital Logic Design']
['11:00am - 2:00pm']
['Hamda Khan']
('room2', 301) 

2
['CS211', 'Discrete Structures']
['11:00am - 2:00pm']
['Sara Aziz']
('room2', 301) 

3
['SE110', 'Intro to Software Engineering']
['12:30pm - 3:30pm']
['Sara Aziz']
('room2', 301) 

Fitness: 0.8 

DETAILS OF THE FIRST EXAM IN POPULATION'S ELEMENT NUMBER 0
Course: ['EE227', 'Digital Logic Design']
Time: ['11:00am - 2:00pm']
Teacher: ['Hamda Khan']
Room: ('room2', 301)
----------------------------------------------------------------
1
['EE227', 'Digital Logic Design']
['9:00am - 12:00pm']
['Aqeel Shahzad']
('room8', 307) 

2
['CS211', 'Discrete Structures']
['1:00pm - 4:00pm']
['Sara Aziz']
('room1', 300) 

3
['SE110', 'Intro to Software Engineering']
['9:00am - 12:00pm']
['Farah Naz']
('room6', 305) 

Fitness: 1.8 

DETAILS OF THE FIRST EXAM IN POPULATION'S ELEMENT NUMBER 1
Course: ['EE227', 'Digital Logic Design']
Time: ['9:00am - 12:00pm']
Teacher: ['Aqeel Shahzad']
Room: ('room8', 307)
--------

In [45]:
class GeneticAlgorithm:

    def crossover_population(self, population):

        # Returning a whole population in which schedules are crossovered

        crossoveredPopulation = Population(0)  # 0 so the schedule list is empty (Initialize() won't be called)

        for i in range(eliteSchedules):
            crossoveredPopulation.get_schedules().append(population.get_schedules()[i])
            # This will append the fittest schedule in the schedules list
            # of the above made class

        # i = 1
        for i in range(1, populationSize):  # to make a new crossovered population

            # return the fittest schedule from the 3 schedules in tournament selection, in s1 and s2
            s1 = self.tournament_selection(population).get_schedules()[0]
            s2 = self.tournament_selection(population).get_schedules()[0]

        print("Tournament:", s1.get_exams()[0].get_teacher())

        # crossoveredPopulation.get_schedules().append(self.crossover_schedule(s1, s2))   # Apply crossover to schedules s1, s2

        # CrossoveredPopulation's schedule list will contain the final schedule

        def crossover_schedule(self, s1, s2):

            crossSchedule = Schedule.initialize()   # Put a schedule in this

            for i in range(0,4):    # The number of courses

                if random.random() > 0.5:
                    crossSchedule.get_exams()[i] = s1.get_exams()[i]
                else:
                    crossSchedule.get_exams()[i] = s2.get_exams()[i]

                return crossSchedule




    def tournament_selection(self, pop):  # Selects two fittest populations

        # Return a list of Schedules. Avoid Plagiarism at all costs.
        tournamentPopulation = Population(0)
        # Size 0 to make empty schedules list, as Population.Initialize() won't be called
        scheduleList = []

        # Takes three random schedules and appends them into list
        # Sort the three schedules and return the list

        for i in range(0, 3):  # 3 schedules are to be selected

            tournamentPopulation.get_schedules().append(pop.get_schedules()[random.randint(0, 4)])

        tournamentPopulation.get_schedules().sort(key=lambda x: x.get_fitness(), reverse=True)
        return tournamentPopulation
        # scheduleList.sort(key = lambda x: get_fitness(), reverse = True)
    # s = Schedule()


In [46]:
# s.initialize()
# print(s.get_exams()[0].get_course(),'\n')

g = GeneticAlgorithm()

l = g.tournament_selection(p)

g.crossover_population(p)

print('\n', p.get_schedules()[0].get_exams()[0].get_teacher())
# Get course of the first exam of the first element in population

Fitness: 1.6 

Fitness: 1.6 

Fitness: 1.8 

Fitness: 0.8 

Fitness: 1.8 

Tournament: ['Usman Rashid']

 ['Hamda Khan']
